In [22]:
import torch
import math
import os
from bvh import Bvh
import csv
import pandas as pd
import numpy as np

1-12(0-11)
Hips.X,Hips.Y,Hips.Z,LeftUpLeg.X,LeftUpLeg.Y,LeftUpLeg.Z,LeftLeg.X,LeftLeg.Y,LeftLeg.Z,LeftFoot.X,LeftFoot.Y,LeftFoot.Z,
<!-- 13-18
LeftToeBase.X,LeftToeBase.Y,LeftToeBase.Z,LeftToeBaseEnd.X,LeftToeBaseEnd.Y,LeftToeBaseEnd.Z,
19-27 -->
(12-20)
RightUpLeg.X,RightUpLeg.Y,RightUpLeg.Z,RightLeg.X,RightLeg.Y,RightLeg.Z,RightFoot.X,RightFoot.Y,RightFoot.Z,
<!-- 28-33
RightToeBase.X,RightToeBase.Y,RightToeBase.Z,RightToeBaseEnd.X,RightToeBaseEnd.Y,RightToeBaseEnd.Z, -->
34-45
Spine.X,Spine.Y,Spine.Z,Spine1.X,Spine1.Y,Spine1.Z,Spine2.X,Spine2.Y,Spine2.Z,Spine3.X,Spine3.Y,Spine3.Z,
46-57(33-44)
LeftShoulder.X,LeftShoulder.Y,LeftShoulder.Z,LeftArm.X,LeftArm.Y,LeftArm.Z,LeftForeArm.X,LeftForeArm.Y,LeftForeArm.Z,LeftHand.X,LeftHand.Y,LeftHand.Z,
<!-- 58-120
LeftHandEnd.X,LeftHandEnd.Y,LeftHandEnd.Z,LeftHandPinky1.X,LeftHandPinky1.Y,LeftHandPinky1.Z,LeftHandPinky2.X,LeftHandPinky2.Y,LeftHandPinky2.Z,LeftHandPinky3.X,LeftHandPinky3.Y,LeftHandPinky3.Z,LeftHandPinky3End.X,LeftHandPinky3End.Y,LeftHandPinky3End.Z,LeftHandRing1.X,LeftHandRing1.Y,LeftHandRing1.Z,LeftHandRing2.X,LeftHandRing2.Y,LeftHandRing2.Z,LeftHandRing3.X,LeftHandRing3.Y,LeftHandRing3.Z,LeftHandRing3End.X,LeftHandRing3End.Y,LeftHandRing3End.Z,LeftHandMiddle1.X,LeftHandMiddle1.Y,LeftHandMiddle1.Z,LeftHandMiddle2.X,LeftHandMiddle2.Y,LeftHandMiddle2.Z,LeftHandMiddle3.X,LeftHandMiddle3.Y,LeftHandMiddle3.Z,LeftHandMiddle3End.X,LeftHandMiddle3End.Y,LeftHandMiddle3End.Z,LeftHandIndex1.X,LeftHandIndex1.Y,LeftHandIndex1.Z,LeftHandIndex2.X,LeftHandIndex2.Y,LeftHandIndex2.Z,LeftHandIndex3.X,LeftHandIndex3.Y,LeftHandIndex3.Z,LeftHandIndex3End.X,LeftHandIndex3End.Y,LeftHandIndex3End.Z,LeftHandThumb1.X,LeftHandThumb1.Y,LeftHandThumb1.Z,LeftHandThumb2.X,LeftHandThumb2.Y,LeftHandThumb2.Z,LeftHandThumb3.X,LeftHandThumb3.Y,LeftHandThumb3.Z,LeftHandThumb3End.X,LeftHandThumb3End.Y,LeftHandThumb3End.Z, -->
121-132
RightShoulder.X,RightShoulder.Y,RightShoulder.Z,RightArm.X,RightArm.Y,RightArm.Z,RightForeArm.X,RightForeArm.Y,RightForeArm.Z,RightHand.X,RightHand.Y,RightHand.Z,
<!-- 133-195
RightHandEnd.X,RightHandEnd.Y,RightHandEnd.Z,RightHandPinky1.X,RightHandPinky1.Y,RightHandPinky1.Z,RightHandPinky2.X,RightHandPinky2.Y,RightHandPinky2.Z,RightHandPinky3.X,RightHandPinky3.Y,RightHandPinky3.Z,RightHandPinky3End.X,RightHandPinky3End.Y,RightHandPinky3End.Z,RightHandRing1.X,RightHandRing1.Y,RightHandRing1.Z,RightHandRing2.X,RightHandRing2.Y,RightHandRing2.Z,RightHandRing3.X,RightHandRing3.Y,RightHandRing3.Z,RightHandRing3End.X,RightHandRing3End.Y,RightHandRing3End.Z,RightHandMiddle1.X,RightHandMiddle1.Y,RightHandMiddle1.Z,RightHandMiddle2.X,RightHandMiddle2.Y,RightHandMiddle2.Z,RightHandMiddle3.X,RightHandMiddle3.Y,RightHandMiddle3.Z,RightHandMiddle3End.X,RightHandMiddle3End.Y,RightHandMiddle3End.Z,RightHandIndex1.X,RightHandIndex1.Y,RightHandIndex1.Z,RightHandIndex2.X,RightHandIndex2.Y,RightHandIndex2.Z,RightHandIndex3.X,RightHandIndex3.Y,RightHandIndex3.Z,RightHandIndex3End.X,RightHandIndex3End.Y,RightHandIndex3End.Z,RightHandThumb1.X,RightHandThumb1.Y,RightHandThumb1.Z,RightHandThumb2.X,RightHandThumb2.Y,RightHandThumb2.Z,RightHandThumb3.X,RightHandThumb3.Y,RightHandThumb3.Z,RightHandThumb3End.X,RightHandThumb3End.Y,RightHandThumb3End.Z, -->
196-204
Neck.X,Neck.Y,Neck.Z,Head.X,Head.Y,Head.Z,
<!-- HeadEnd.X, HeadEnd.Y,HeadEnd.Z -->

In [26]:

csv_path = "/home/liumin/litianyi/workspace/data/datasets/traj/0213_take_06_worldpos.csv"
df = pd.read_csv(csv_path,usecols=[1,2,3,4,5,6,7,8,9,10,11,12,19,20,21,22,23,24,25,26,27,34,35,36,37,38,
                                   39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,121,122,123,
                                   124,125,126,127,128,129,130,131,132,196,197,198,199,200,201,])
step = 4
keypoints = df.head(1).values.squeeze(0)
keypoint = np.concatenate([keypoints[0:3],keypoints[-6:],keypoints[45:57],keypoints[33:45],keypoints[12:21],
                      keypoints[3:12]])
print(keypoint.shape)
print(keypoint)
from utils.visualize import DrawSkeleton
DrawSkeleton(keypoint)



(51,)
[ -1.8198     -85.1331      32.3154      -2.29266051 -84.54505034
  52.06092621  -2.52217607 -83.35151635  56.20908567  -1.07942106
 -84.46957603  49.64075706   4.08146813 -85.73506579  48.79305632
   6.97904736 -85.09099628  40.20234025   9.11834891 -82.78314827
  31.34676945  -3.19806528 -84.02186922  49.56264904  -8.22112172
 -82.99083668  47.93147951 -11.18611181 -83.15283364  39.34114865
 -11.22018819 -81.16073864  30.15668873   1.25729818 -85.83009467
  31.19381954   4.2670253  -79.9468586   16.99771846   2.33735625
 -84.63750604   3.87485452  -4.9968338  -84.39044245  31.56227747
  -3.86754621 -83.88145268  15.95247771  -1.87107343 -85.85549278
   2.1665205 ]


/root/autodl-tmp/home/liumin/litianyi/workspace/project/Ego-Pose-Estimation/utils/visualize.py:44: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  ax.scatter3D(xp, yp, zp, cmap='Greens')
